**개요 : 음식 설명에 대한 피쳐 벡터화를 진행하고 코사인 유사도를 계산해 음식 설명이 비슷한 음식을 추천해준다.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**food1과 food2를 최대한 합쳐보자**

In [2]:
import pandas as pd
import numpy as np

df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/food1.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/food2.csv')

In [3]:
df1.head()

,식품코드,식품명,데이터구분코드,데이터구분명,식품기원코드,식품기원명,식품대분류코드,식품대분류명,대표식품코드,대표식품명,...,출처코드,출처명,1회 섭취참고량,외식업체명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자,제공기관코드,제공기관명
0,D105-007000000-0001,김치국,D,음식,1,가정식(분석 함량),5,국 및 탕류,5007,김치국,...,3,식품의약품안전처,400.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처
1,D105-007280000-0001,김치국_콩나물,D,음식,1,가정식(분석 함량),5,국 및 탕류,5007,김치국,...,3,식품의약품안전처,400.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처
2,D105-010000000-0001,내장탕,D,음식,1,가정식(분석 함량),5,국 및 탕류,5010,내장탕,...,3,식품의약품안전처,600.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처
3,D105-011100000-0001,냉국_미역,D,음식,1,가정식(분석 함량),5,국 및 탕류,5011,냉국,...,3,식품의약품안전처,350.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처
4,D105-011100100-0001,냉국_미역_오이,D,음식,1,가정식(분석 함량),5,국 및 탕류,5011,냉국,...,3,식품의약품안전처,350.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처


In [4]:
df2.head()

,구분,음식명,영문,음식설명
0,면,가락국수,Thick Noodles in Clear Broth,"Noodles in a clear broth, topped with vegetabl..."
1,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...
2,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...
3,어류,가자미,Sole,NaN
4,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to..."


In [5]:
print(df1.shape, df2.shape)

(1328, 50) (751, 4)


In [6]:
df1.rename(columns = {'대표식품명' : '음식명'}, inplace=True)

In [7]:
df1.head(3)

,식품코드,식품명,데이터구분코드,데이터구분명,식품기원코드,식품기원명,식품대분류코드,식품대분류명,대표식품코드,음식명,...,출처코드,출처명,1회 섭취참고량,외식업체명,데이터생성방법코드,데이터생성방법명,데이터생성일자,데이터기준일자,제공기관코드,제공기관명
0,D105-007000000-0001,김치국,D,음식,1,가정식(분석 함량),5,국 및 탕류,5007,김치국,...,3,식품의약품안전처,400.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처
1,D105-007280000-0001,김치국_콩나물,D,음식,1,가정식(분석 함량),5,국 및 탕류,5007,김치국,...,3,식품의약품안전처,400.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처
2,D105-010000000-0001,내장탕,D,음식,1,가정식(분석 함량),5,국 및 탕류,5010,내장탕,...,3,식품의약품안전처,600.0,해당없음,1,분석,2020-03-31,2022-06-28,1471000,식품의약품안전처


In [8]:
df2 = df2.merge(df1[['식품명', '식품대분류명', '음식명']], on='음식명')

In [9]:
df2.head(3)

,구분,음식명,영문,음식설명,식품명,식품대분류명
0,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...,가래떡,"곡류, 서류 제품"
1,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...,가오리찜,찜류
2,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to...",가자미구이,구이류


In [10]:
df2.head(20)

,구분,음식명,영문,음식설명,식품명,식품대분류명
0,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...,가래떡,"곡류, 서류 제품"
1,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...,가오리찜,찜류
2,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to...",가자미구이,구이류
3,젓갈,가자미식해,Spicy Fermented Sole,A spicy dish of sole mixed with cooked millet ...,가자미식해,장아찌·절임류
4,조림,가자미조림,Braised Sole,"Sole seasoned with soy sauce, sugar, green oni...",가자미조림,조림류
5,"전,적",가지전,Pan-fried Eggplant,"Eggplant slices coated with flour, dipped in e...",가지전,전·적 및 부침류
6,구이,갈비구이,Marinated and Grilled Beef Short Ribs,Beef short ribs marinated in a mixture of soy ...,갈비구이_쇠고기,구이류
7,구이,갈비구이,Marinated and Grilled Beef Short Ribs,Beef short ribs marinated in a mixture of soy ...,갈비구이_쇠고기_양념,구이류
8,구이,갈비구이,Marinated and Grilled Beef Short Ribs,Beef short ribs marinated in a mixture of soy ...,갈비구이_왕갈비_양념,구이류
9,찜,갈비찜,Braised Short Ribs,"Beef short ribs cut into chunks, seasoned with...",갈비찜_쇠고기,찜류


In [11]:
df2.shape

(361, 6)

In [12]:
df2.to_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/new_food.csv')

**food2 의 음식 설명만 가지고 음식 추천 예측결과 확인**

In [13]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MachineLearningPractice/Colab-Notebooks/FarmKit/foodfile/food2.csv')

In [14]:
df.head()

,구분,음식명,영문,음식설명
0,면,가락국수,Thick Noodles in Clear Broth,"Noodles in a clear broth, topped with vegetabl..."
1,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...
2,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...
3,어류,가자미,Sole,NaN
4,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to..."


In [15]:
df['음식설명']

0      Noodles in a clear broth, topped with vegetabl...
1      Cylindrical rice pasta sliced diagonally into ...
2      Marinated stingray steamed and served with thi...
3                                                    NaN
4      Sole seasoned with salt then grilled, often to...
                             ...                        
746    Hamheung-style buckwheat noodles mixed with a ...
747    Slices of raw fish and fresh vegetables served...
748    A porridge made of white rice and ground black...
749    Rice cake made from white rice flour. The rice...
750    A porridge made of white rice only. The rice i...
Name: 음식설명, Length: 751, dtype: object

In [16]:
df = df.dropna()    # 결손 데이터 처리

In [17]:
df.shape

(691, 4)

In [18]:
df.reset_index(inplace=True)
df.drop('index', inplace=True, axis=1)
df.head(10)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,구분,음식명,영문,음식설명
0,면,가락국수,Thick Noodles in Clear Broth,"Noodles in a clear broth, topped with vegetabl..."
1,떡,가래떡,Cylindrical Rice Pasta,Cylindrical rice pasta sliced diagonally into ...
2,찜,가오리찜,Steamed Stingray,Marinated stingray steamed and served with thi...
3,구이,가자미구이,Grilled Sole,"Sole seasoned with salt then grilled, often to..."
4,젓갈,가자미식해,Spicy Fermented Sole,A spicy dish of sole mixed with cooked millet ...
5,조림,가자미조림,Braised Sole,"Sole seasoned with soy sauce, sugar, green oni..."
6,나물,가지나물,Eggplant Namul,"Eggplant steamed then marinated in soy sauce, ..."
7,선,가지선,Stuffed Eggplant,"Eggplants cut lengthwise, filled with seasonin..."
8,"전,적",가지전,Pan-fried Eggplant,"Eggplant slices coated with flour, dipped in e..."
9,찜,가지찜,Stuffed Eggplant with Beef,"Eggplant cut lengthwise, filled with seasoned ..."


In [19]:
df['음식설명'].isnull().values.any()    # 결손 데이터 없는 것 확인

False

**피처 벡터화**   
1. BOW 피처 벡터화 - CountVectorizer   
https://pearlluck.tistory.com/682
2. BOW 피처 벡터화 - TfidVectorizer   
https://pearlluck.tistory.com/683

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')     # 크게 의미없는 단어들을 제외해준다.
tfidf_matrix = tfidf.fit_transform(df['음식설명'])
tfidf_matrix.shape

(691, 1252)

In [31]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)     # 코사인 유사도 계산
cosine_sim.shape

(691, 691)

In [32]:
cosine_sim

array([[1.        , 0.04921025, 0.17323513, ..., 0.        , 0.        ,
        0.        ],
       [0.04921025, 1.        , 0.04083023, ..., 0.02338472, 0.22212268,
        0.06050846],
       [0.17323513, 0.04083023, 1.        , ..., 0.        , 0.04323272,
        0.        ],
       ...,
       [0.        , 0.02338472, 0.        , ..., 1.        , 0.10518675,
        0.18514917],
       [0.        , 0.22212268, 0.04323272, ..., 0.10518675, 1.        ,
        0.22126028],
       [0.        , 0.06050846, 0.        , ..., 0.18514917, 0.22126028,
        1.        ]])

In [22]:
indices = pd.Series(df.index, index=df['음식명']).drop_duplicates()    # 중복 제거
indices

음식명
가락국수       0
가래떡        1
가오리찜       2
가자미구이      3
가자미식해      4
        ... 
회냉면      686
회덮밥      687
흑임자죽     688
흰떡       689
흰죽       690
Length: 691, dtype: int64

In [23]:
# 음식명을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 음식 목록 반환
def get_recommendations(title, cosine_sim=cosine_sim):

  # 음식명을 통해서 전체 데이터 기준 그 음식의 index 값을 얻기
  idx = indices[title]

  # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 코사인 유사도 기준으로 내림차순 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 자기 자신을 제외한 10개의 추천 음식을 슬라이싱
  sim_scores = sim_scores[1:11]

  # 추천 음식목록 10개의 인덱스 정보 추출
  movie_indices = [i[0] for i in sim_scores]

  # 인덱스 정보를 통해 음식명 추출
  return df['음식명'].iloc[movie_indices]

In [24]:
get_recommendations('가락국수')

88      김치냉면
292      버섯국
518     잔치국수
362       소면
23       감잣국
374    쇠고기산적
547     조개국수
367     송이볶음
183     동태찌개
325     비빔국수
Name: 음식명, dtype: object

In [25]:
get_recommendations('된장찌개')

195          된장국
591        청국장찌개
488       우렁된장찌개
285          배춧국
576           찌개
486    우거짓국/우거지탕
598          추어탕
284        배추속댓국
418      아귀탕/아구탕
656       해물된장찌개
Name: 음식명, dtype: object

In [26]:
get_recommendations('가지찜')

385       수육
7        가지선
512      잉어찜
643       편육
381       수란
363    소박이김치
647      표고전
642       편수
73      국수장국
72        국수
Name: 음식명, dtype: object

In [27]:
get_recommendations('약과')

522      잣강정
333       산자
588     채소튀김
377    쇠고기튀김
81       굴튀김
343     새우튀김
496    유(밀)과
27        강정
307       부각
439       연근
Name: 음식명, dtype: object